Implementation of LASER to the corpus

In [1]:
# Imports
import numpy as np
from scipy import spatial
import numpy as np
import pandas as pd
import time

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import keras
from keras import models
from keras import layers


Using TensorFlow backend.


In [2]:
# Parameters

# laser model to use
laser_model = "de_en_laser_model__testcorr_0.3574_MAE_0.6445_time28_05_14_10.h5"

Load Dataset

In [3]:
df1 = pd.read_csv("scores.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28404 entries, 0 to 28403
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   28404 non-null  int64  
 1   source       28404 non-null  object 
 2   reference    28404 non-null  object 
 3   translation  28404 non-null  object 
 4   metric       28404 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.1+ MB


In [5]:
df1.head()

,Unnamed: 0,source,reference,translation,metric
0,0,Das Publikum ist fast gleichmäßig zwischen Sch...,The audience is almost evenly split between bl...,The audience is almost evenly split between bl...,2.764301
1,1,Du kannst ihre Energie durch den Bildschirm sp...,"You can feel their energy through the screen. """"","You can feel her energy through the screen.""",1.287924
2,2,"Da die Adresse unbekannt ist, wird die Mithilf...","As the address is unknown, the help of the pop...","As the address is unknown, the assistance of t...",1.499944
3,3,"Arsenal-Manager Arsene Wenger, dessen Verein i...","Arsenal manager Arsene Wenger, whose club is o...","Arsenal manager Arsene Wenger, whose club is o...",2.764301
4,4,Landwirtschaftsminister im Interview - Wie sch...,Agriculture Minister in the interview - How do...,Minister of Agriculture in interview – How do ...,0.204494


In [6]:
# For this one, we need to drop the unnamed column 0
# df1 = df1.drop(labels="Unnamed: 0",axis=1)

In [7]:
# Verification
# Check for empty or sparse reference / translation, and drop them.
for column in ["source","reference","translation"]:
    print(column)
    bad_idx = [idx for idx in np.where(df1[column].str.len()<=2)[0]]
    if bad_idx != []:
        print(df1.iloc[bad_idx])
    print(f"Bad idx: {bad_idx}")
#    df1 = df1.drop(index=bad_idx)

source
Bad idx: []
reference
Bad idx: []
translation
Bad idx: []


Loading LASER embeddings

In [8]:
source_arr = np.load("laser.source_embeds.npy")
refer_arr = np.load("laser.reference_embeds.npy")
trans_arr = np.load("laser.translation_embeds.npy")

Cosine Similarity

In [9]:
cos_similarity_ref_hyp = []
cos_similarity_src_ref = []
cos_similarity_src_hyp = []

# for each of the cos similarity, put them into lists.
for i in range(refer_arr.shape[0]):
    cos_similarity_ref_hyp.append((spatial.distance.cosine(refer_arr[i],trans_arr[i])*-1)+1)
    cos_similarity_src_ref.append((spatial.distance.cosine(source_arr[i],refer_arr[i])*-1)+1)
    cos_similarity_src_hyp.append((spatial.distance.cosine(source_arr[i],trans_arr[i])*-1)+1)


# Standardize and transform into series.
scaler = StandardScaler()
cs_rh = pd.Series(scaler.fit_transform(np.array(cos_similarity_ref_hyp).reshape(-1,1)).flatten(),name="cos_sim_ref_hyp")
cs_sr = pd.Series(scaler.fit_transform(np.array(cos_similarity_src_ref).reshape(-1,1)).flatten(),name="cos_sim_src_ref")
cs_sh = pd.Series(scaler.fit_transform(np.array(cos_similarity_src_hyp).reshape(-1,1)).flatten(),name="cos_sim_src_hyp")



In [10]:
# Z scores of Cosine Similarity.
cos_df = pd.concat([cs_rh,cs_sr,cs_sh],axis=1)
cos_df

,cos_sim_ref_hyp,cos_sim_src_ref,cos_sim_src_hyp
0,1.375766,0.261304,0.401669
1,0.892460,0.253938,0.403947
2,1.288441,0.166684,0.304615
3,1.375768,0.773871,0.924371
4,0.270158,0.789520,0.451771
...,...,...,...
28399,0.451945,0.500903,0.528470
28400,-0.248215,-0.059200,0.336520
28401,0.498707,0.841823,0.864680
28402,-2.303439,-0.935509,-1.079092


---
Using Neural Networks, directly on the embeddings.

In [11]:
# Loading the model.
model = keras.models.load_model(laser_model)

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [13]:
# Prepare the input layer.
full_arr_c = np.concatenate((refer_arr,trans_arr),axis=1)
X = full_arr_c.copy()

In [14]:
df1["metric"] = model.predict(X)

In [16]:
df1.to_csv("scores.csv",index=False)